In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite:///../database/investment_data.sqlite")

In [3]:

comp = pd.read_sql("SELECT * FROM compustat_quarterly", con=engine)
comp['datadate'] = pd.to_datetime(comp['datadate'])
crsp = pd.read_sql("SELECT * FROM crsp_daily", con=engine)
crsp['date'] = pd.to_datetime(crsp['date'])
ff = pd.read_sql("SELECT * FROM fama_french_daily", con=engine)

crsp['quarter'] = crsp['date'].dt.to_period('Q')
returns_qtr = crsp.groupby(['symbol', 'quarter'])['ret'].apply(lambda x: (1 + x).prod() - 1).reset_index()
returns_qtr = returns_qtr.rename(columns={'ret': 'qtr_return'})

comp['quarter'] = comp['datadate'].dt.to_period('Q')

merged = pd.merge(comp, returns_qtr, on=['symbol', 'quarter'], how='inner')
merged['quarter'] = merged['quarter'].astype(str)
merged.to_sql("merged_fundamentals_returns", con=engine, if_exists="replace", index=False)
print("Merged data with quarterly returns stored in database.")


Merged data with quarterly returns stored in database.


In [4]:
print("Export to MySQL skipped. All data remains in SQLite database.")

Export to MySQL skipped. All data remains in SQLite database.
